# Big Data Analytics Project 

In [5]:
import pandas as pd

# Load the JSON file into a Pandas DataFrame
salesdf = pd.read_json('sales.json', lines=True)


In [6]:
!pip install pandas

In [7]:
num_rows, num_columns = salesdf.shape

print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

Number of Rows: 5000
Number of Columns: 7
